In [2]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

import numpy as np
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

In [2]:
# !pip install tensorflow-privacy

In [3]:
import tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.analysis.compute_dp_sgd_privacy_lib import compute_dp_sgd_privacy_statement

In [4]:
import pandas as pd
df = pd.read_csv('athletes.csv')

In [5]:
df.head()

,region,gender,age,height,weight,candj,snatch,deadlift,backsq,eat,...,eat_conv,eat_cheat,eat_quality,eat_paleo,eat_weigh,deadlift_norm,candj_norm,snatch_norm,backsq_norm,total_lift
0,USA,1,30.0,71.0,200.0,235.0,175.0,385.0,315.0,I eat whatever is convenient|,...,1,0,0,0,0,1.925000,1.175000,0.875000,1.575000,5.550000
1,Africa,1,28.0,70.0,176.0,187.0,134.0,335.0,254.0,I eat 1-3 full cheat meals per week|,...,0,1,0,0,0,1.903409,1.062500,0.761364,1.443182,5.170455
2,USA,1,35.0,68.0,225.0,285.0,205.0,440.0,405.0,I eat quality foods but don't measure the amount|,...,0,0,1,0,0,1.955556,1.266667,0.911111,1.800000,5.933333
3,USA,1,36.0,71.0,199.0,267.0,212.0,485.0,390.0,I eat quality foods but don't measure the amount|,...,0,0,1,0,0,2.437186,1.341709,1.065327,1.959799,6.804020
4,USA,1,36.0,64.0,155.0,245.0,180.0,415.0,385.0,I eat strict Paleo|,...,0,0,0,1,0,2.677419,1.580645,1.161290,2.483871,7.903226


In [6]:
drop_cols = ['eat', 'background', 'experience', 'schedule', 'howlong', 'height', 'weight', 'candj', 'snatch', 'deadlift',
       'backsq', 'candj_norm', 'snatch_norm', 'backsq_norm', 'deadlift_norm']
df.drop(drop_cols, axis=1, inplace=True)

In [7]:
df = pd.get_dummies(df, columns=['region'], dtype=int)


In [8]:
df

,gender,age,BMI,rec_sports,high_sports,col_sports,pro_sports,no_bg,exp_coach,exp_alone,...,eat_paleo,eat_weigh,total_lift,region_Africa,region_Asia,region_Australia,region_Canada,region_Europe,region_Latin America,region_USA
0,1,30.0,27.894029,1,1,1,0,0,0,1,...,0,0,5.550000,0,0,0,0,0,0,1
1,1,28.0,25.253091,0,0,0,0,1,1,0,...,0,0,5.170455,1,0,0,0,0,0,0
2,1,35.0,34.210754,0,1,0,0,0,1,0,...,0,0,5.933333,0,0,0,0,0,0,1
3,1,36.0,27.754559,0,1,1,0,0,1,0,...,0,0,6.804020,0,0,0,0,0,0,1
4,1,36.0,26.605395,1,1,1,0,0,0,1,...,1,0,7.903226,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29417,1,25.0,23.387806,0,0,0,0,1,1,0,...,0,0,4.723926,0,0,0,0,0,0,1
29418,1,24.0,24.966124,1,1,1,0,0,1,0,...,0,0,5.183908,0,0,1,0,0,0,0
29419,0,25.0,21.627611,1,1,1,0,0,0,1,...,0,0,4.896825,0,0,0,0,0,1,0
29420,0,22.0,23.598381,1,1,1,0,0,1,0,...,0,0,2.873563,0,0,0,0,0,0,1


In [9]:
from sklearn.model_selection import train_test_split
X = df.drop('total_lift', axis = 1)
y = df.total_lift
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=123 )

In [22]:
y_train = np.array(y_train)
y_test = np.array(y_test)
X_test = np.array(X_test)
X_train = np.array(X_train)

In [23]:
#define hyperparameters
epochs = 10
batch_size = 128

In [24]:
l2_norm_clip = 1.5 #A value of 0 means that no noise is added to the model parameters
noise_multiplier = 1.2 # A value of 0 means that no noise is added to the gradients
num_microbatches = 1 #A value of 1 means that the gradients are computed using the entire training set
learning_rate = 0.05 #A value of 0 means that the model parameters will not be updated

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [25]:
input_dim = X_train.shape[1]

In [26]:
model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Dense(units=64, activation='relu', input_shape=(input_dim,)))
# model.add(tf.keras.layers.Dense(units=32, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='linear'))

In [27]:
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

loss = tf.keras.losses.MeanSquaredError()


In [28]:
model.compile(optimizer=optimizer, loss=loss, metrics=['mae'])

model.fit(X_train, y_train,
          epochs=epochs,
          validation_data=(X_test, y_test),
          batch_size=batch_size)

Train on 20595 samples, validate on 8827 samples
Epoch 1/10
20595/20595 [==============================] - 0s 23us/sample - loss: 33.9726 - mean_absolute_error: 4.5824 - val_loss: 28.5415 - val_mean_absolute_error: 4.3605
Epoch 2/10
20595/20595 [==============================] - 0s 13us/sample - loss: 35.4633 - mean_absolute_error: 4.6875 - val_loss: 14.5296 - val_mean_absolute_error: 3.0246
Epoch 3/10
20595/20595 [==============================] - 0s 12us/sample - loss: 46.2428 - mean_absolute_error: 5.3897 - val_loss: 30.7093 - val_mean_absolute_error: 4.4361
Epoch 4/10
20595/20595 [==============================] - 0s 12us/sample - loss: 47.1954 - mean_absolute_error: 5.4687 - val_loss: 40.9487 - val_mean_absolute_error: 5.3103
Epoch 5/10
20595/20595 [==============================] - 0s 12us/sample - loss: 46.2939 - mean_absolute_error: 5.4230 - val_loss: 66.3209 - val_mean_absolute_error: 6.7666
Epoch 6/10
20595/20595 [==============================] - 0s 12us/sample - loss: 40.89

In [30]:
loss_dp, mae_dp = model.evaluate(X_test, y_test)

# Print the performance measures
print("Test Loss:", loss_dp)
print("Test MAE:", mae_dp)

Test Loss: 109.23550374675096
Test MAE: 8.796069


In [18]:
privacy_guarantee = compute_dp_sgd_privacy_statement(number_of_examples=X_train.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              num_epochs=epochs,
                                              delta=1e-5)

print("Privacy Guarantee Epsilon: \n", privacy_guarantee)

Privacy Guarantee Epsilon: 
 DP-SGD performed over 20595 examples with 128 examples per iteration, noise
multiplier 1.2 for 10 epochs with microbatching, and no bound on number of
examples per user.

This privacy guarantee protects the release of all model checkpoints in addition
to the final model.

Example-level DP with add-or-remove-one adjacency at delta = 1e-05 computed with
RDP accounting:
    Epsilon with each example occurring once per epoch:        37.721
    Epsilon assuming Poisson sampling (*):                      6.999

No user-level privacy guarantee is possible without a bound on the number of
examples per user.

(*) Poisson sampling is not usually done in training pipelines, but assuming
that the data was randomly shuffled, it is believed the actual epsilon should be
closer to this value than the conservative assumption of an arbitrary data
order.



In [20]:
tensorflow_privacy.compute_dp_sgd_privacy(n=X_train.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

(1.116550308561137, 13.0)

From the above results from the `compute_dp_sgd_privacy_statement` we can see that after assuming each record appears once per epoch, the epsilon value is 37.721, while the value of epsilon is 6.999 after assuming poisson sampling. This is because the epsilon value is an estimation assuming a randomly shuffled data.

The value has also been calculates using the `compute_dp_sgd_privacy` function and it assumes poisson subsampling but does not accounnt for sensitivity and micro batching which led to its deprecation. The values obtained for the epsilon (privacy guarantee) is 1.1165 and the second number 13 indicates that an example can be included in the training process for a maximum of 13 times.

---



## Comparing with Non DP NN for performance measures


In [31]:
model.compile(loss='mae', optimizer='sgd', metrics=['mae'])

In [32]:
model.fit(X_train, y_train, epochs=10, batch_size=250)


Train on 20595 samples
Epoch 1/10
20595/20595 [==============================] - 0s 8us/sample - loss: 6.2533 - mean_absolute_error: 6.2533
Epoch 2/10
20595/20595 [==============================] - 0s 5us/sample - loss: 6.2956 - mean_absolute_error: 6.2956
Epoch 3/10
20595/20595 [==============================] - 0s 5us/sample - loss: 6.7389 - mean_absolute_error: 6.7389
Epoch 4/10
20595/20595 [==============================] - 0s 6us/sample - loss: 6.7617 - mean_absolute_error: 6.7617
Epoch 5/10
20595/20595 [==============================] - 0s 5us/sample - loss: 7.0557 - mean_absolute_error: 7.0557
Epoch 6/10
20595/20595 [==============================] - 0s 4us/sample - loss: 6.8541 - mean_absolute_error: 6.8541
Epoch 7/10
20595/20595 [==============================] - 0s 6us/sample - loss: 7.1779 - mean_absolute_error: 7.1779
Epoch 8/10
20595/20595 [==============================] - 0s 5us/sample - loss: 7.1333 - mean_absolute_error: 7.1333
Epoch 9/10
20595/20595 [=================

In [34]:
loss_nondp, mae_nondp = model.evaluate(X_test, y_test)

# Print the performance measures
print("Test Loss with no DP:", loss_nondp)
print("Test MAE with no DP:", mae_nondp)

Test Loss with no DP: 7.377688575306848
Test MAE with no DP: 7.3776884
